In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# FAISS tutorial
# https://medium.com/loopio-tech/how-to-use-faiss-to-build-your-first-similarity-search-bf0f708aa772

documents = [
    "Medical Services: Immunizations, Allergies, Travel vaccinations, Laboratory testing, Medical Clinics, Medical Records.",
    "Mental Health: Counseling, therapy sessions, crisis intervention, mental wellness programs.",
    "Pharmacy: Prescription refills, medication counseling, and pharmaceutical services.",
    "Radiology: Diagnostic imaging services including X-rays, MRIs, and CT scans.",
    "Women's Health: exams, maternity care, and reproductive health services."
]

# Load transformer
model = SentenceTransformer('all-MiniLM-L6-v2') 
# other transformer options include: "paraphrase-mpnet-basev2" ...

# Generate embeddings
embeddings = model.encode(documents)
embeddings = np.array(embeddings).astype('float32')

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# similarity search
def search(query, k=3):
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, k)
    return distances, indices

# query
query = "I need help with a prescription refill"
distances, indices = search(query)

print("Query:", query)
print("\nResults:")
for rank, idx in enumerate(indices[0], start=1):
    print(f"Rank {rank}:")
    print(f"Category: {documents[idx]}")
    print(f"Distance: {distances[0][rank-1]:.4f}\n")

Query: I need help with a prescription refill

Results:
Rank 1:
Category: Pharmacy: Prescription refills, medication counseling, and pharmaceutical services.
Distance: 0.7573

Rank 2:
Category: Medical Services: Immunizations, Allergies, Travel vaccinations, Laboratory testing, Medical Clinics, Medical Records.
Distance: 1.6369

Rank 3:
Category: Women's Health: exams, maternity care, and reproductive health services.
Distance: 1.8260



In [ ]:
# Extended Test Suite

test_queries = [
    "I need help with a prescription refill",                # Direct match to Pharmacy
    "Where can I get my medications renewed?",               # Similar to prescription refill
    "What mental health services are available?",            # Direct match to Mental Health
    "I want to talk to someone about my anxiety",            # Semantic match for mental health counseling
    "Do you offer immunizations and allergy tests?",         # Direct match to Medical Services
    "How do I get vaccinated for travel?",                   # Semantic match to immunizations/travel vaccinations
    "What are the clinic hours for the lab?",                # Semantic match for Laboratory testing
    "Can I schedule a gynecological exam?",                  # Direct match to Women's Health
    "I'm looking for help with medication counseling",       # Direct match to Pharmacy services
    "I need to get my lab work done",                        # Semantic match for Medical Clinics/Laboratory
]

for query in test_queries:
    distances, indices = search(query)
    print("Query:", query)
    print("Results:")
    for rank, idx in enumerate(indices[0], start=1):
        print(f"Rank {rank}:")
        print(f"Category: {documents[idx]}")
        print(f"Distance: {distances[0][rank-1]:.4f}")
    print("-" * 50)

Query: I need help with a prescription refill
Results:
Rank 1:
Category: Pharmacy: Prescription refills, medication counseling, and pharmaceutical services.
Distance: 0.7573
Rank 2:
Category: Medical Services: Immunizations, Allergies, Travel vaccinations, Laboratory testing, Medical Clinics, Medical Records.
Distance: 1.6369
Rank 3:
Category: Women's Health: exams, maternity care, and reproductive health services.
Distance: 1.8260
--------------------------------------------------
Query: Where can I get my medications renewed?
Results:
Rank 1:
Category: Pharmacy: Prescription refills, medication counseling, and pharmaceutical services.
Distance: 0.9502
Rank 2:
Category: Medical Services: Immunizations, Allergies, Travel vaccinations, Laboratory testing, Medical Clinics, Medical Records.
Distance: 1.4581
Rank 3:
Category: Mental Health: Counseling, therapy sessions, crisis intervention, mental wellness programs.
Distance: 1.5184
--------------------------------------------------
Query: